# C20 Team: Aryamaaan Jena, Hemaang Kotta, Shubham Agarwal, Maneet Singh

In [58]:
import numpy as np
import pandas as pd
import seaborn as sb
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from arch.univariate import GARCH, EWMAVariance 
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import norm
import scipy
from arch import arch_model

Reading DataFrame

In [59]:
signals = pd.read_excel("/Users/hemaangkotta/Downloads/Portfolio/HW7/gmo_analysis_data.xlsx", sheet_name = 'signals')
signals = signals.set_index('Date')
r_total = pd.read_excel("/Users/hemaangkotta/Downloads/Portfolio/HW7/gmo_analysis_data.xlsx", sheet_name = 'returns (total)')
r_total = r_total.set_index('Date')
rf = pd.read_excel("/Users/hemaangkotta/Downloads/Portfolio/HW7/gmo_analysis_data.xlsx", sheet_name = 'risk-free rate')
rf = rf.set_index('Date')
r_excess = r_total.subtract(rf.US3M,axis='rows')

Summary Stats

In [46]:
def summary_stats(df, annual_fac):
    report = pd.DataFrame()
    report['Mean'] = df.mean() * annual_fac
    report['Vol'] = df.std() * np.sqrt(annual_fac)
    report['Sharpe'] = report['Mean'] / report['Vol']
    return round(report, 4)

In [54]:
r_excess = r_excess.dropna()

### 2.1.Calculate the mean, volatility, and Sharpe ratio for GMWAX. Do this for three samples:• from inception through 2011• 2012-present• inception - present. Has the mean, vol, and Sharpe changed much since the case?

In [71]:
s2011 = summary_stats(r_excess.loc[:"2011",["GMWAX"]],12)
s2011 = s2011.rename(index={"GMWAX": "Inception-2011"})
s2012 = summary_stats(r_excess.loc["2012":,["GMWAX"]],12)
s2012 = s2012.rename(index={"GMWAX": "2012-Present"})
s_all = summary_stats(r_excess[["GMWAX"]],12)
s_all = s_all.rename(index={"GMWAX": "Inception-Present"})
summary  = pd.concat([s2011,s2012,s_all])
summary

,Mean,Vol,Sharpe
Inception-2011,0.0158,0.1250,0.1266
2012-Present,0.0593,0.0853,0.6952
Inception-Present,0.0329,0.1111,0.2964


Yes the Mean, Vol and Sharpe did change, the Mean excess returns increased from 1.58% to 5.93% and Vol has decreased from 12.50% to 8.53%

### 2.2 GMO believes a risk premium is compensation for a security's tendency to lose money at bad times". For all three samples, analyze extreme scenarios by looking at Min return, 5th percentile (VaR-5th),Maximum drawdown1

In [191]:
def summary_stats2(df,df2, annual_fac):
    report = pd.DataFrame()
    report['Min Return'] = df.min()
    report['VaR'] = df.quantile(0.05)
    wealth_index = 1000*(1+df2).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdown = (wealth_index - previous_peaks)/previous_peaks
    report["Drawdown"]= drawdown.min()
    return round(report, 4)

In [192]:
s2011 = summary_stats2(r_excess.loc[:"2011",["GMWAX"]],r_total.loc[:"2011",["GMWAX"]],12)
s2011 = s2011.rename(index={"GMWAX": "Inception-2011"})
s2012 = summary_stats2(r_excess.loc["2012":,["GMWAX"]],r_total.loc["2012":,["GMWAX"]],12)
s2012 = s2012.rename(index={"GMWAX": "2012-Present"})
s_all = summary_stats2(r_excess[["GMWAX"]],r_total[["GMWAX"]],12)
s_all = s_all.rename(index={"GMWAX": "Inception-Present"})
summary2  = pd.concat([s2011,s2012,s_all])
summary2

,Min Return,VaR,Drawdown
Inception-2011,-0.1492,-0.0598,-0.3552
2012-Present,-0.1187,-0.0306,-0.1675
Inception-Present,-0.1492,-0.0449,-0.3552


In [193]:
summary3 = summary.join(summary2)
summary3

,Mean,Vol,Sharpe,Min Return,VaR,Drawdown
Inception-2011,0.0158,0.1250,0.1266,-0.1492,-0.0598,-0.3552
2012-Present,0.0593,0.0853,0.6952,-0.1187,-0.0306,-0.1675
Inception-Present,0.0329,0.1111,0.2964,-0.1492,-0.0449,-0.3552


(a) Does GMWAX have high or low tail-risk as seen by these stats?

Yes it has high tail risks.It has a VaR of 5.98% for a mean returns of 1.58%

(b)Does that vary much across the two subsamples?

VaR has decreased from 5.98% to 3.06% in the two sub samples.

### For all three samples, regress excess returns of GMWAX on excess returns of SPY.

(a)Report the estimated alpha, beta, and r-squared.

In [183]:
def regression(y_df,x_df,time):
    reg_stats = pd.DataFrame(data = None)
    # Drop the NAs in y
    y = y_df.dropna()
    # Align the X with y 
    X = sm.add_constant(x_df.loc[y.index])
    reg = sm.OLS(y, X).fit()
    reg_stats.loc[time,'alpha'] = reg.params[0]*12
    reg_stats.loc[time,'beta'] = reg.params[1]
    reg_stats.loc[time,"R-Squared"]=reg.rsquared
    reg_stats.loc[time,"IR"]=reg.params[0]*pow(12,0.5)/reg.resid.std()
    return reg_stats

In [109]:
a1=regression(r_excess.GMWAX.loc[:"2011"],r_excess.SPY.loc[:"2011"],"Inception-2011")
a2=regression(r_excess.GMWAX.loc["2012":],r_excess.SPY.loc["2012":],"2012-Present")
a3=regression(r_excess.GMWAX,r_excess.SPY,"Inception-Present")
a = pd.concat([a1,a2,a3])
a

,alpha,beta,R-Squared
Inception-2011,-0.005751,0.539615,0.507129
2012-Present,-0.028521,0.568297,0.763293
Inception-Present,-0.013511,0.546054,0.566804


(b) Is GMWAX a low-beta strategy? Has that changed since the case?

Yes, the beta is less. No the beta has not increased since the case as well.

(c) Does GMWAX provide alpha? Has that changed across the subsamples?

The Alpha is negative. It doesn't provide any positive alpha.The Alpha has changed from -0.575% to -2.85%

### Q3)Forecast Regressions
Consider the lagged regression, where the regressor, (X,) is a period behind the target, (rSPY ).

In [148]:
signals_shifted= signals.shift(1).dropna()
signals_shifted.head()

,DP,EP,US10Y
Date,,,
1993-03-31,2.82,4.44,6.03
1993-04-30,2.77,4.41,6.03
1993-05-31,2.82,4.44,6.05
1993-06-30,2.81,4.38,6.16
1993-07-31,2.79,4.31,5.80


In [149]:
SPY_forecast = r_total[["SPY"]].loc["1993-03-31":]

In [160]:
predict =pd.DataFrame(data=None)
def regression2(y_df,x_df,name):
    reg_stats = pd.DataFrame(data = None)
    # Drop the NAs in y
    y = y_df.dropna()
    # Align the X with y 
    X = sm.add_constant(x_df.loc[y.index])
    reg = sm.OLS(y, X).fit()
    reg_stats.loc[name,"R-Squared"]=reg.rsquared
    predict[name]= reg.predict(X)
    return reg_stats

In [161]:
r1 = regression2(SPY_forecast,signals_shifted[["DP"]],"DP")
r2 = regression2(SPY_forecast,signals_shifted[["EP"]],"EP")
r3 = regression2(SPY_forecast,signals_shifted[["DP","EP","US10Y"]],"3 Factors")
r4 = pd.concat([r1,r2,r3])
r4

,R-Squared
DP,0.006830
EP,0.007773
3 Factors,0.015022


In [172]:
strategy =(predict*100).multiply(r_total['SPY'].loc["1993-03-31":],axis=0)
strategy["SPY"]= r_total['SPY'].loc["1993-03-31":]
strategy.head()

,DP,EP,3 Factors,SPY
Date,,,,
1993-03-31,0.036656,0.020071,0.026936,0.022408
1993-04-30,-0.040850,-0.022695,-0.029744,-0.025589
1993-05-31,0.044119,0.024157,0.032349,0.026970
1993-06-30,0.005970,0.003220,0.004267,0.003667
1993-07-31,-0.007828,-0.004164,-0.005729,-0.004855


In [256]:
def summary_stats(df, annual_fac):
    report = pd.DataFrame()
    report['Mean'] = df.mean() * annual_fac
    report['Vol'] = df.std() * np.sqrt(annual_fac)
    report['Sharpe'] = report['Mean'] / report['Vol']
    report['VaR'] = df.quantile(0.05)
    wealth_index = 1000*(1+df).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdown = (wealth_index - previous_peaks)/previous_peaks
    report["Drawdown"]= drawdown.min()
    return round(report, 4)

In [199]:
summary_stats(strategy, 12)

,Mean,Vol,Sharpe,VaR,Drawdown
DP,0.1180,0.1511,0.7809,-0.0579,-0.6415
EP,0.1200,0.1321,0.9081,-0.0549,-0.3850
3 Factors,0.1354,0.1499,0.9032,-0.0645,-0.5193
SPY,0.1114,0.1462,0.7620,-0.0694,-0.5080


In [260]:
b1 = regression(strategy.DP,strategy.SPY.dropna(),"DP")
b2 = regression(strategy.EP,strategy.SPY,"EP")
b3 = regression(strategy["3 Factors"],strategy.SPY,"3 Factors")
b4 = pd.concat([b1,b2,b3])
b4

,alpha,beta,R-Squared,IR
DP,0.013172,0.940695,0.828754,0.210708
EP,0.031071,0.798050,0.779833,0.501224
3 Factors,0.040830,0.849104,0.685469,0.485577


### GMO believes a risk premium is compensation for a security's tendency to lose money at bad times". Let's consider risk characteristics.

(a)For both strategies, the market, and GMO, calculate the monthly VaR for   = :05. Just
use the quantile of the historic data for this VaR calculation.

The VaR is highest for the 3 Factor Model prediction using DP,EP and US10Y

(b)The GMO case mentions that stocks under-performed short-term bonds from 2000-2011.
Does the dynamic portfolio above under-perform the risk-free rate over this time?

In [205]:
strategy["Rf"]= rf['US3M'].loc["1993-03-31":]
strategy.loc["2000":"2011"].mean()*12

DP           0.036507
EP           0.036712
3 Factors    0.059687
SPY          0.018159
Rf           0.023062
dtype: float64

No the Dynamic portfolio doesn't underform Rf. The returns are greater than Risk free rate.

(c)Based on the regression estimates, in how many periods do we estimate a negative risk
premium?

In [237]:
strategy["Excess"]= np.where(strategy["3 Factors"]<strategy.Rf,True,False)

In [238]:
len(strategy[strategy["Excess"]==True])/len(strategy["Excess"])

0.375

Do you believe the dynamic strategy takes on extra risk??

No, the dynamic stategy has the highest Sharpe ratio. It doesn't take on extra risk.

### 4 Out-of-Sample Forecasting

In [243]:
X =sm.add_constant(signals_shifted.EP)
Y =SPY_forecast

### Initialize the error lists 
err_x = np.array([])
err_null = np.array([])
predOOS =np.array([])
for i in range(60, len(X)):
    ### Data up to t
    curr_X = X.iloc[:i]
    curr_y = Y.iloc[:i]
    
    ### Fit the model on data through t-1
    model =sm.OLS(curr_y, curr_X, missing='drop').fit()
   

    ### Reshape the matrix so we can input it to predict
    predict_y = model.predict(X.iloc[i].values.reshape(1, -1))[0]
    
    ### Actual OOS value
    actual_y = y.iloc[i]
    
    ### The expanding mean is our baseline for OOS r2, curr_y has data from time 1 to time t-1
    expanding_mean = curr_y.mean()
    
    ### Append the errors, which we will use to calculate OOS r2
    err_x = np.append(err_x, actual_y - predict_y)
    err_null = np.append(err_null, actual_y - expanding_mean)
    predOOS = np.append(predOOS,predict_y)

In [244]:
r_sqr_oos = 1 - np.square(err_x).sum() / np.square(err_null).sum()
print('OOS r-squared: ' + str(round(r_sqr_oos, 4)))

OOS r-squared: -0.0074


Did this forecasting strategy produce a positive OOS r-squared?
No it doesn't produce a positive OOS R Squared. It's infact negative 0.0074

### 4.2. Re-do problem 3.2 using this OOS forecast. How much better/worse is the OOS Earnings-Price ratio strategy compared to the in-sample version of 3.2? 

In [268]:
predOOSdf = pd.DataFrame(predOOS,index=r_total['SPY'].iloc[61:].index,columns=['predOOS'])
wt_predOOS = (predOOSdf*100).multiply(r_total['SPY'].loc["1998-03-31":],axis=0)
wt_predOOS["SPY"]=r_total['SPY'].loc["1998-03-31":]
wt_predOOS

,predOOS,SPY
Date,,
1998-03-31,0.045644,0.048758
1998-04-30,0.014350,0.012791
1998-05-31,-0.022508,-0.020769
1998-06-30,0.035103,0.042591
1998-07-31,-0.014576,-0.013514
...,...,...
2021-06-30,0.013377,0.022473
2021-07-31,0.004363,0.024412
2021-08-31,0.019158,0.029760


In [269]:
summary_stats(wt_predOOS, 12)

,Mean,Vol,Sharpe,VaR,Drawdown
predOOS,0.0870,0.1633,0.5331,-0.0671,-0.5837
SPY,0.0924,0.1526,0.6054,-0.0745,-0.5080


In [261]:
bOOS = regression(wt_predOOS["predOOS"],strategy.SPY,"predOOS")
b4 = pd.concat([b3,bOOS])
b4

,alpha,beta,R-Squared,IR
3 Factors,0.040830,0.849104,0.685469,0.485577
predOOS,0.040106,0.508105,0.225378,0.279084


The OOS performs worse than the insample 3 Factor model. Infact the Information ratio for insample is 48% while the OOS is 27.9%

### 4.3. Re-do problem 3.3 using this OOS forecast. Is the point-in-time version of the strategy riskier?

The VaR is highest for the OOS forecast only.VaR is -6.71% for OOS while 3 factors is -6.45%

(b)The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

In [263]:
wt_predOOS["Rf"]= rf['US3M'].loc["1998-03-31":]
wt_predOOS.loc["2000":"2011"].mean()*12

predOOS    0.038768
SPY        0.018159
Rf         0.023062
dtype: float64

The dynamic portfolio does out-perform Rf.

Based on the regression estimates, in how many periods do we estimate a negative risk
premium?

In [265]:
wt_predOOS["Excess"]= np.where(wt_predOOS["predOOS"]<wt_predOOS.Rf,True,False)
len(wt_predOOS[wt_predOOS["Excess"]==True])/len(wt_predOOS["Excess"])

0.3767605633802817

37.67% times we are less than Risk free return

(d)Do you believe the dynamic strategy takes on extra risk??

In [270]:
summary_stats(wt_predOOS, 12) 

,Mean,Vol,Sharpe,VaR,Drawdown
predOOS,0.0870,0.1633,0.5331,-0.0671,-0.5837
SPY,0.0924,0.1526,0.6054,-0.0745,-0.5080


Yes, the strategy takes on extra risk.The Volatility for OOS is 16.33% wheras SPY is 15.26%. Even the Sharpe is lower for OOS.